In [4]:
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import sys
sys.path.append('..')

import json
import logging
import sys
import os
import math
import random
import shutil
from pathlib import Path
from itertools import chain
# from megatron.data.dataset_utils import get_indexed_dataset_

# import horovod.torch as hvd
# from dotenv import load_dotenv
import torch
import numpy as np
import datasets
from torch.utils.data import DataLoader, DistributedSampler
from datasets import Dataset, load_dataset, load_from_disk
from huggingface_hub import hf_hub_download
from sklearn.metrics import f1_score, accuracy_score

from lm_experiments_tools import TrainerArgs
from lm_experiments_tools.trainer import Trainer

from torch.nn.utils.rnn import pad_sequence
from lm_experiments_tools.lm_datasets import get_lm_datasets

### Finetune

In [10]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
import random 

num_mem_tokens = 2
max_n_segments = 4
vary_n_segments = False
block_size = 128 - 2*2
input_seq_len = 248
history_size = input_seq_len - block_size

class segmentDataLoaderOTF(DataLoader):
    def __init__(self, dataset, block_size, history_size, max_samples=None, shuffle=False, *args, **kwargs):
        super().__init__(dataset, *args, **kwargs)
        self.block_size = block_size
        self.history_size = history_size
        self.max_samples = max_samples
        self.shuffle = shuffle
            
    def get_samples(self, document):
        input_ids, attention_mask = document['input_ids'], document['attention_mask']
        samples = [input_ids[max({0, start - self.history_size}): start + self.block_size] for start in range(0, len(input_ids), self.block_size)]
        return samples
    
    def __iter__(self):
        inds = list(range(len(self.dataset)))

        if self.max_samples is not None:
            inds = inds[:self.max_samples]

        if self.shuffle: 
            random.shuffle(inds)

        doc_ind = 0
        samples = []

        while True:
            if doc_ind >= len(inds):
                break
            try: 
                while len(samples) < self.batch_size:
                    document = self.dataset[inds[doc_ind]]
                    doc_ind += 1
                    samples += self.get_samples(document)
                    if doc_ind >= len(inds):
                        raise(StopIteration)
            except(StopIteration):
                pass

            batch, samples = samples[:self.batch_size], samples[self.batch_size:]
            yield self.collate_fn(batch)
        


In [13]:

from torch.nn.utils.rnn import pad_sequence
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
def collate_fn(batch):
    input_ids = labels = [torch.tensor(b[::-1]) for b in batch]
    attention_mask = [torch.ones_like(b, dtype=int) for b in input_ids]
    input_ids = pad_sequence(input_ids, padding_value=id_pad_value).T.flip(1)
    labels = pad_sequence(labels, padding_value=-100).T.flip(1)
    attention_mask = pad_sequence(attention_mask, padding_value=0).T.flip(1)

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'attention_mask': attention_mask}
    
    if input_ids.shape[1] != block_size:
        labels_mask = torch.ones_like(input_ids, dtype=bool)
        labels_mask[:, :-block_size] = False
        collated['labels_mask'] = labels_mask

    # if getattr(args, 'vary_n_segments', False):
    if vary_n_segments:
        n_segments = np.random.randint(1, max_n_segments + 1)
        n_tokens = n_segments * block_size
        for k in collated:
            collated[k] = collated[k][:, -n_tokens:]
    
    return collated

In [122]:
from datasets import Dataset, load_dataset, load_from_disk

vary_n_segments = False

valid_dataset = load_from_disk('/home/jovyan/rmt/datasets/arxiv/valid')
valid_dataloader = segmentDataLoaderOTF(valid_dataset, 
                                    batch_size=2, 
                                    # sampler=valid_sampler,
                                    block_size=block_size, 
                                    history_size=history_size, 
                                    shuffle=False,
                                    max_samples=1000,
                                    collate_fn=collate_fn, drop_last=True)#, **kwargs)

In [123]:
gen = iter(valid_dataloader)

In [151]:
batch = next(gen)
batch['input_ids'].shape

torch.Size([2, 248])

In [125]:
batch = next(gen)
batch['input_ids'].shape

torch.Size([2, 248])

In [142]:
batch = next(gen)
batch['input_ids'].shape

torch.Size([2, 248])

# -----------------------------------------------------------------------------------

In [4]:
# num_segments = 1
# num_mem_tokens = 2
# device = torch.device(3)
device = 'cpu'

In [5]:
from transformers import AutoModel

base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [6]:
# base_model.state_dict()

In [7]:


# tokenizer = AutoTokenizer.from_pretrained(model_name)

# rmt_config = {'num_mem_tokens': 1, 
#                 'max_n_segments': 5,
#                #  'segment_alignment': 'right',
#                 'tokenizer': tokenizer,
#                #  'memory_layers': 'all', 
#                #  'share_memory_layers': True,
#                #  'reconstruction_loss_coef': 0.1,
#                 'offset_position_ids': True,
#                 'k1': -1, 'k2': 3,
#                 'segment_ordering': 'regular',
#                 'input_size': 128, 
#                 'bptt_depth': -1, 
#                 'sum_loss': False,
#              }

# # base_model = AutoModelForCausalLM.from_pretrained(model_name)
# rmt = RMTDecoderLMHeadMultiSeg(base_model, **rmt_config)
# # rmt2 = RMTEncoderForTokenClassification(base_model, **rmt_config)

# # base_model3 = AutoModelForCausalLM.from_pretrained(model_name)
# # rmt_offset = RMTDecoderLMHead(base_model3, **rmt_config)


# 籴

In [8]:
# for n, p in rmt.named_parameters():
#     if 'memory' in n:
#         print(n)

In [9]:
# rmt.eval()
# rmt.train()
# 1

In [240]:
input_ids = batch['input_ids'].clone()#[:, :512]
labels = batch['labels'].clone()#[:, :512]

labels_mask = batch['labels_mask']
# rmt_out_0 = rmt(input_ids, labels=labels, labels_mask=labels_mask, output_hidden_states=True, output_attentions=True)
# rmt_out.loss
kwargs = dict(**batch)

In [242]:
rmt_out = rmt(**kwargs)

In [243]:
[(key, rmt_out[key]) for key in rmt_out if 'loss' in key]

[('loss', tensor(2.7514, grad_fn=<NllLossBackward>)),
 ('loss_0', tensor(2.5994, grad_fn=<NllLossBackward>)),
 ('loss_1', tensor(2.7056, grad_fn=<NllLossBackward>)),
 ('loss_2', tensor(2.5862, grad_fn=<NllLossBackward>)),
 ('loss_3', tensor(2.7514, grad_fn=<NllLossBackward>))]

In [ ]:
[(key, rmt_out[key]) for key in rmt_out if 'loss' in key]

[('loss', tensor(11.3409, grad_fn=<NllLossBackward>)),
 ('loss_0', tensor(2.9731, grad_fn=<NllLossBackward>)),
 ('loss_1', tensor(2.6483, grad_fn=<NllLossBackward>)),
 ('loss_2', tensor(2.8106, grad_fn=<NllLossBackward>)),
 ('loss_3', tensor(2.6017, grad_fn=<NllLossBackward>))]

In [186]:
cpt_path = "../../runs/lm_long/wikitext-2-v1/gpt2/lr5e-05_linear_adamw_wd1e-03_630-128-5x128_mem1_bs32_regular_bptt-5_from_cpt_4-5/run_1/model_best.pth"
cpt = torch.load(cpt_path, map_location='cpu')
rmt.load_state_dict(cpt['model_state_dict'])

<All keys matched successfully>

In [154]:
# input_ids = valid_batch['input_ids'].clone()#[:, :512]
# labels = valid_batch['labels'].clone()#[:, :512]
# labels_mask = None
# kwargs = dict(**valid_batch)
# kwargs['labels_mask'] = None

In [155]:
input_ids.shape

torch.Size([2, 432])

In [170]:
def detach_memory(self, seg_num):
    k2, max_n_segments = self.rmt_config.get('k2'), self.rmt_config.get('max_n_segments')
    if seg_num == 0 \
        or k2 in {-1, None} \
        or seg_num + k2 > max_n_segments:
        return False
    return True

In [171]:
self = rmt
memory = self.set_memory(input_ids.shape)
segmented = self.pad_and_segment(input_ids, labels)

base_model_outputs = []
for seg_num, segment in enumerate(zip(*segmented)):                

    seg_kwargs, non_empty_mask = self.prepare_kwargs(segment, memory, kwargs)
    if sum(non_empty_mask) == 0:
        # raise(ValueError('nonemptymask is zero'))
        continue
    
    print('seg_num, k2, detach_memory', seg_num, self.rmt_config.get('k2'), detach_memory(self, seg_num))
    if detach_memory(self, seg_num):
        memory = memory.detach()

    seg_kwargs['inputs_embeds'][:, self.read_memory_position] = memory[non_empty_mask]
    seg_kwargs['inputs_embeds'][:, self.write_memory_position] = memory[non_empty_mask]
    out = self.model(**seg_kwargs)
    base_model_outputs.append(out)
    
    memory[non_empty_mask] = out.hidden_states[-1][:, self.write_memory_position]

self.memory_state = memory
# return (base_model_outputs, kwargs)
out = self.process_outputs(base_model_outputs, kwargs)
# return out

seg_num, k2, detach_memory 0 3 False
seg_num, k2, detach_memory 1 3 True
seg_num, k2, detach_memory 2 3 False
seg_num, k2, detach_memory 3 3 False


In [172]:
out.loss.backward()

In [53]:
memory.shape

torch.Size([2, 10, 768])

In [175]:
base_model_outputs[0].loss, base_model_outputs[1].loss, base_model_outputs[2].loss, base_model_outputs[3].loss

(tensor(4.9269, grad_fn=<NllLossBackward>),
 tensor(10.3476, grad_fn=<NllLossBackward>),
 tensor(10.0047, grad_fn=<NllLossBackward>),
 tensor(11.6276, grad_fn=<NllLossBackward>))

In [127]:
segments = [torch.ones(8) * i for i in range(1, 4)]
memory = torch.nn.Parameter(torch.ones(2) * 5, requires_grad=True)
ws = [torch.nn.Parameter(torch.ones(10) * 2, requires_grad=True)] * 3
# w = torch.nn.Parameter(torch.ones(10) * 2, requires_grad=True)

In [128]:
mem = memory.repeat(1)

outputs = []
losses = []
k2 = 1
for i, (seg, w) in enumerate(zip(segments, ws)):
    print('i, k2, max_n_segments', i,  k2, len(segments),  i + k2 <= len(segments) and i != 0)
    if i + k2 <= len(segments) and i != 0:
        mem = mem.detach()
        # inp = torch.cat((mem.detach(), seg))
    # else:
        # inp = torch.cat((mem, seg))
    inp = torch.cat((mem, seg))
    out = inp.T + w
    mem = out[:2]
    loss = out.mean()
    
    outputs.append(out)
    losses.append(loss)

total_loss = torch.cat(outputs)[-10:].mean()

i, k2, max_n_segments 0 1 3 False
i, k2, max_n_segments 1 1 3 True
i, k2, max_n_segments 2 1 3 True


In [129]:
[w.grad for w in ws]

[None, None, None]

In [130]:
total_loss.backward()

In [120]:
# k2 = 3
[w.grad for w in ws], memory.grad

([tensor([0.3000, 0.3000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000]),
  tensor([0.3000, 0.3000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000]),
  tensor([0.3000, 0.3000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000])],
 tensor([0.1000, 0.1000]))

In [126]:
# k2 = 2
[w.grad for w in ws], memory.grad

([tensor([0.2000, 0.2000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000]),
  tensor([0.2000, 0.2000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000]),
  tensor([0.2000, 0.2000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000])],
 tensor([0., 0.]))

In [131]:
# k2 = 1
[w.grad for w in ws], memory.grad

([tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000]),
  tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000]),
  tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000])],
 tensor([0., 0.]))

In [248]:
# # k2 = 0
# [w.grad for w in ws]

In [196]:
mem.grad, memory.grad

/tmp/ipykernel_1871387/516110708.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  mem.grad, memory.grad


(None, tensor([0.1000, 0.1000]))

In [162]:
segments

[tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([2., 2., 2., 2., 2., 2., 2., 2.]),
 tensor([3., 3., 3., 3., 3., 3., 3., 3.])]

In [159]:
ws[0].shape

torch.Size([10])

In [114]:
rmt_out.keys()

odict_keys(['loss', 'logits', 'hidden_states', 'loss_0', 'logits_0', 'hidden_states_0', 'attentions_0'])

In [115]:
rmt_out.logits.shape, len(rmt_out.hidden_states), rmt_out.hidden_states[0].shape

(torch.Size([1, 51, 50257]), 13, torch.Size([1, 51, 768]))

### load dataset 

In [234]:
class Holder:
    def __init__(self):
        pass
    
max_n_segments = 4
input_size = 128
num_mem_tokens = 1

input_seq_len = target_seq_len = max_n_segments * (input_size - 2 * num_mem_tokens)
batch_size = 2

args = Holder
args.max_n_segments = max_n_segments
args.target_seq_len = target_seq_len
args.input_seq_len = input_seq_len
args.input_prefix = ''
args.block_size = None

device = 'cpu'

### Decoder

In [235]:
from lm_experiments_tools.lm_datasets import *
raw_datasets = datasets.load_dataset('wikitext', 'wikitext-2-v1')
# train_dataset, _ = get_lm_datasets(raw_datasets, tokenizer, block_size=args.input_seq_len)
# _, valid_dataset = get_lm_datasets(raw_datasets, tokenizer, block_size=input_size - 2 * num_mem_tokens)

/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
Found cached dataset wikitext (/home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [236]:
block_size = input_size - 2 * num_mem_tokens
history_size = args.input_seq_len - block_size

column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)

def group_texts(examples, block_size, history_size=None):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    if history_size is None:
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    else:
        result = {
            k: [t[max({0, i - history_size}) : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    result["labels"] = result["input_ids"].copy()
    return result


train_dataset = tokenized_datasets["train"].map(lambda x: group_texts(x, block_size, history_size), 
                                        batched=True, desc=f"Grouping train in chunks of {block_size} and history {history_size}")
valid_dataset = tokenized_datasets["validation"].map(lambda x: group_texts(x, block_size), 
                                        batched=True, desc=f"Grouping valid in chunks of {block_size}")
test_dataset = tokenized_datasets["test"].map(lambda x: group_texts(x, block_size), 
                                        batched=True, desc=f"Grouping test in chunks of {block_size}")

Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-389b922bfc5fe729.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-6067a66e735cfbb1.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-941845a5470f2db7.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-b4e1ff576225fa59.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-3f80d227394578e9.a

In [237]:
len(train_dataset)

19244

In [215]:
len(train_dataset[0]['input_ids']), len(valid_dataset[0]['input_ids']), len(test_dataset[0]['input_ids'])

(126, 126, 126)

In [216]:
len(train_dataset[1]['input_ids']), len(valid_dataset[1]['input_ids']), len(test_dataset[1]['input_ids'])

(252, 126, 126)

In [217]:
len(train_dataset[2]['input_ids']), len(valid_dataset[2]['input_ids']), len(test_dataset[2]['input_ids'])

(378, 126, 126)

In [218]:
len(train_dataset[-1]['input_ids']), len(valid_dataset[-1]['input_ids']), len(test_dataset[-1]['input_ids'])

(385, 37, 8)

In [219]:
from torch.nn.utils.rnn import pad_sequence
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
def collate_fn(batch):
    input_ids = [torch.tensor(b['input_ids'][::-1]) for b in batch]
    labels = [torch.tensor(b['labels'][::-1]) for b in batch]
    attention_mask = [torch.tensor(b['attention_mask'][::-1]) for b in batch]
    input_ids = pad_sequence(input_ids, padding_value=id_pad_value).T.flip(1)
    labels = pad_sequence(labels, padding_value=-100).T.flip(1)
    attention_mask = pad_sequence(attention_mask, padding_value=0).T.flip(1)

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'attention_mask': attention_mask}
    
    if input_ids.shape[1] != block_size:
        labels_mask = torch.ones_like(input_ids, dtype=bool)
        labels_mask[:, :-block_size] = False
        collated['labels_mask'] = labels_mask
    
    return collated

In [220]:
# dataloader for RMT
# batch sample i is a continuation of sample i of the previous batch
class alignedDataLoader(DataLoader):
    def __iter__(self):
        # all_inds = np.arange(len(self.dataset))
        all_inds = np.arange(len(self.dataset) // self.batch_size * batch_size)
        all_inds = all_inds.reshape(batch_size, -1)
        for batch_ind in range(all_inds.shape[1]):
            batch = [self.dataset[int(ind)] for ind in all_inds[:, batch_ind]]
            yield self.collate_fn(batch)


kwargs = {'pin_memory': True, 'num_workers': 1}

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, 
                                # sampler=train_sampler,
                                collate_fn=collate_fn, **kwargs)

valid_dataloader = alignedDataLoader(valid_dataset, batch_size=batch_size, 
                            # sampler=valid_sampler,
                                collate_fn=collate_fn, drop_last=True, **kwargs)

In [221]:
src = [train_dataset[i] for i in range(2)]

In [223]:
gen = iter(train_dataloader)
batch = next(gen)

In [238]:
batch = next(gen)
batch = next(gen)
batch = next(gen)


In [31]:
valid_gen = iter(valid_dataloader)
valid_batch = next(valid_gen)

In [36]:
valid_batch['input_ids'].shape

torch.Size([2, 108])

In [26]:
batch['labels']

tensor([[ -100,  -100,  -100,  ...,   416,  5609,   511],
        [  796,   569, 18354,  ...,  2478,  2233,   284]])

In [27]:
batch['labels'][:, 1004:]

tensor([[  796,   569, 18354,  ...,   416,  5609,   511],
        [ 8686,  4282,   764,  ...,  2478,  2233,   284]])

In [25]:
batch['labels_mask'][:, 1004:]

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [119]:
tokenizer.decode(batch['input_ids'][:, 1004:][0])

' = Valkyria Chronicles III = \n Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3, lit. Valkyria of the Battlefield 3 ), commonly referred to as Valkyria Chronicles III outside Japan, is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable. Released in January 2011 in Japan, it is the third game in the Valkyria series. <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors, the story runs parallel to the first game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more <unk> for series newcomers. Character designer <

In [53]:
batch = next(gen)


In [54]:
batch['input_ids']

tensor([[8686, 4282,  764,  ...,  262, 2656,  837],
        [ 262, 3859, 1445,  ..., 3670,  373, 7867]])

In [162]:
gen = iter(valid_dataloader)
batch = next(gen)

In [163]:
tokenizer.decode(batch['input_ids'][0][-50:])

'-@ eastern Atlantic Ocean from northern Norway to the Azores and Morocco, not including the Baltic Sea. It is also present in most of the Mediterranean Sea, only missing from the section east of Crete, and along only the north @-@'

In [164]:
tokenizer.decode(batch['input_ids'][1][-50:])

' the <unk> and <unk> powers of nearly every one, <unk> their opponents... \n The side left <unk> and travelled north where they played <unk> in Masterton. The match was won 10 – 8, and'

In [165]:
batch = next(gen)

In [166]:
tokenizer.decode(batch['input_ids'][0][:50])

' west coast of the Black Sea. The <unk> populations are found in the Norwegian <unk> <unk> and <unk>, inside the Arctic Circle. \n The species can be divided into four genetically distinct populations, one widespread population,'

In [168]:
tokenizer.decode(batch['input_ids'][1][:50])

' the next day they faced Wellington, who they also defeated. The fixture against Wellington was nearly abandoned because Scott and the Wellington Rugby Union could not agree on a venue ; the match went ahead only when the Wellington officials agreed to cede the <unk>'

In [21]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [127]:
batch['input_ids'][0][:10]

tensor([  796,   569, 18354,  7496, 17740,  6711,   796,   220,   198,  2311])

In [128]:
batch['labels'][0][:10]

tensor([  796,   569, 18354,  7496, 17740,  6711,   796,   220,   198,  2311])

In [130]:
train_dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [131]:
train_dataset[0]['input_ids'][:10]

[796, 569, 18354, 7496, 17740, 6711, 796, 220, 198, 2311]

In [132]:
train_dataset[0]['labels'][:10]

[796, 569, 18354, 7496, 17740, 6711, 796, 220, 198, 2311]

In [18]:
batch['input_ids'].shape

torch.Size([2, 2008])

In [19]:
tokenizer.decode(batch['input_ids'][0])

' = Valkyria Chronicles III = \n Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3, lit. Valkyria of the Battlefield 3 ), commonly referred to as Valkyria Chronicles III outside Japan, is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable. Released in January 2011 in Japan, it is the third game in the Valkyria series. <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors, the story runs parallel to the first game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more <unk> for series newcomers. Character designer <

In [77]:
for batch in valid_dataloader:
    if batch['input_ids'].shape[0] != 2:
        print(batch['input_ids'].shape)

In [13]:
batch.keys(), batch['input_ids'].shape

(dict_keys(['input_ids', 'attention_mask', 'labels']), torch.Size([2, 1014]))

In [21]:
out = base_model(**batch)

KeyboardInterrupt: 